In [2]:
!pip install -U 'hopsworks[python]' --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.5/640.5 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install requests-cache retry_requests openmeteo-requests --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.6 MB/s eta 0:00:00


In [137]:
pip install scikit-learn==1.3.1

In [220]:
import hopsworks

project = hopsworks.login(api_key_value = "fGaRronKJ6ZMI6K0.4iXEy9yDd6VkxOypTtfseQQ1Ip3a9sREDzgx6Qnezj50mhqfD7DrzfBlpQPMFAuM")
fs = project.get_feature_store()

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1209521


In [136]:
from sklearn.ensemble import RandomForestRegressor


In [221]:
# Get or create the 'transactions' feature group
air_quality_fg = fs.get_or_create_feature_group(
    name="aqi_weather_features",
    version=2
)

In [119]:
selected_features = air_quality_fg.select_all()

In [120]:
selected_features.show(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 


,main_aqi,date,temperature_2m,relative_humidity_2m,precipitation,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,...,week_of_year,is_weekend,hour,aqi_lag_1,temp_lag_1,aqi_lag_2,temp_lag_2,aqi_lag_3,temp_lag_3,index
0,3,2024-09-25 14:00:00+00:00,28.450001,74.376190,0.0,0.0,1003.299988,1002.391357,17.987394,251.927582,...,39,0,14,3.0,29.100000,3.0,29.750000,3.0,30.600000,14352
1,5,2024-10-27 02:00:00+00:00,24.549999,91.371178,0.0,0.0,1009.900024,1008.973389,6.090812,325.840240,...,43,1,2,4.0,25.600000,4.0,26.799999,4.0,27.299999,15108
2,5,2023-11-28 21:00:00+00:00,16.049999,79.478691,0.0,0.0,1015.299988,1014.341125,9.885262,33.111355,...,48,0,21,5.0,16.450001,5.0,17.150000,5.0,18.049999,7255
3,5,2024-01-02 11:00:00+00:00,23.549999,50.575592,0.0,0.0,1014.599976,1013.665894,21.758419,231.044083,...,1,0,11,5.0,25.100000,5.0,25.500000,5.0,24.700001,8085
4,3,2024-07-18 17:00:00+00:00,30.500000,80.913979,0.0,0.0,998.200012,997.301941,19.191748,246.801392,...,29,0,17,3.0,30.600000,4.0,31.150000,4.0,31.299999,12771


In [121]:
feature_view = fs.get_or_create_feature_view(
    name='aqi_fv',
    version=1,
    query=selected_features,
    labels=["main_aqi"]
)

In [122]:
feature_group = fs.get_or_create_feature_group(
    name="aqi_weather_features",
    version=2
)

In [123]:
df = feature_group.read()
df.sort_values(by = 'date', inplace=True)
df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.51s) 


,main_aqi,date,temperature_2m,relative_humidity_2m,precipitation,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,...,week_of_year,is_weekend,hour,aqi_lag_1,temp_lag_1,aqi_lag_2,temp_lag_2,aqi_lag_3,temp_lag_3,index
11323,2,2023-01-24 14:00:00+00:00,17.450001,33.484470,0.0,0.0,1016.200012,1015.244751,11.901798,273.468170,...,4,0,14,2.0,17.450001,2.0,17.450001,2.0,17.450001,0
5559,2,2023-01-24 15:00:00+00:00,16.299999,40.659458,0.0,0.0,1016.700012,1015.740417,9.360001,292.619904,...,4,0,15,2.0,17.450001,2.0,17.450001,2.0,17.450001,1
12626,4,2023-01-24 16:00:00+00:00,14.900000,51.539116,0.0,0.0,1017.099976,1016.135498,7.729527,297.758423,...,4,0,16,2.0,16.299999,2.0,17.450001,2.0,17.450001,2
1279,5,2023-01-24 17:00:00+00:00,14.150000,64.489677,0.0,0.0,1017.299988,1016.332825,8.699793,294.443939,...,4,0,17,4.0,14.900000,2.0,16.299999,2.0,17.450001,3
8414,5,2023-01-24 18:00:00+00:00,13.500000,68.425240,0.0,0.0,1017.400024,1016.430542,9.686609,311.987122,...,4,0,18,5.0,14.150000,4.0,14.900000,2.0,16.299999,4


In [331]:
TEST_SIZE = 0.2

X_train, X_test, y_train, y_test = feature_view.train_test_split(
    test_size=TEST_SIZE,
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.58s) 


In [332]:
# Sort the training features DataFrame 'X_train' based on the 'datetime' column
X_train = X_train.sort_values("index")

# Reindex the target variable 'y_train' to match the sorted order of 'X_train' index
y_train = y_train.reindex(X_train.index)

In [333]:
X_train

,date,temperature_2m,relative_humidity_2m,precipitation,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,...,week_of_year,is_weekend,hour,aqi_lag_1,temp_lag_1,aqi_lag_2,temp_lag_2,aqi_lag_3,temp_lag_3,index
11323,2023-01-24 14:00:00+00:00,17.450001,33.484470,0.0,0.0,1016.200012,1015.244751,11.901798,273.468170,20.519999,...,4,0,14,2.0,17.450001,2.0,17.450001,2.0,17.450001,0
5559,2023-01-24 15:00:00+00:00,16.299999,40.659458,0.0,0.0,1016.700012,1015.740417,9.360001,292.619904,15.840000,...,4,0,15,2.0,17.450001,2.0,17.450001,2.0,17.450001,1
1279,2023-01-24 17:00:00+00:00,14.150000,64.489677,0.0,0.0,1017.299988,1016.332825,8.699793,294.443939,10.799999,...,4,0,17,4.0,14.900000,2.0,16.299999,2.0,17.450001,3
8414,2023-01-24 18:00:00+00:00,13.500000,68.425240,0.0,0.0,1017.400024,1016.430542,9.686609,311.987122,11.879999,...,4,0,18,5.0,14.150000,4.0,14.900000,2.0,16.299999,4
11820,2023-01-24 19:00:00+00:00,12.950000,65.559967,0.0,0.0,1017.099976,1016.128845,9.931042,313.531250,11.879999,...,4,0,19,5.0,13.500000,5.0,14.150000,4.0,14.900000,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15268,2025-01-22 15:00:00+00:00,21.208500,59.000000,0.0,0.0,1014.400024,1013.458740,9.904906,289.093597,21.959999,...,4,0,15,5.0,21.208500,5.0,22.158501,5.0,22.158501,17209
8746,2025-01-22 16:00:00+00:00,20.758501,62.000000,0.0,0.0,1015.099976,1014.156494,7.244860,296.564972,18.000000,...,4,0,16,5.0,20.758501,5.0,21.208500,5.0,21.208500,17210
10630,2025-01-22 17:00:00+00:00,19.658501,65.000000,0.0,0.0,1015.900024,1014.952271,6.725354,15.524177,12.959999,...,4,0,17,5.0,19.658501,5.0,20.758501,5.0,20.758501,17211
6860,2025-01-22 21:00:00+00:00,18.158501,55.000000,0.0,0.0,1016.000000,1015.047485,4.510787,28.610369,7.920000,...,4,0,21,5.0,18.158501,4.0,18.858500,4.0,18.858500,17215


In [334]:
# Sort the test features DataFrame 'X_test' based on the 'datetime' column
X_test = X_test.sort_values("index")

# Reindex the target variable 'y_test' to match the sorted order of 'X_test' index
y_test = y_test.reindex(X_test.index)

In [335]:
X_test

,date,temperature_2m,relative_humidity_2m,precipitation,rain,pressure_msl,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m,...,week_of_year,is_weekend,hour,aqi_lag_1,temp_lag_1,aqi_lag_2,temp_lag_2,aqi_lag_3,temp_lag_3,index
12626,2023-01-24 16:00:00+00:00,14.900000,51.539116,0.0,0.0,1017.099976,1016.135498,7.729527,297.758423,11.879999,...,4,0,16,2.0,16.299999,2.0,17.450001,2.0,17.450001,2
5411,2023-01-24 23:00:00+00:00,10.550000,40.891636,0.0,0.0,1016.500000,1015.521240,16.039202,350.960602,24.480000,...,4,0,23,5.0,11.450000,5.0,13.350000,5.0,12.650000,9
8118,2023-01-25 04:00:00+00:00,11.950000,38.244236,0.0,0.0,1019.000000,1018.023926,11.570515,5.355733,19.440001,...,4,0,4,5.0,9.950000,5.0,9.300000,5.0,9.550000,14
5326,2023-01-25 06:00:00+00:00,16.750000,25.848072,0.0,0.0,1019.700012,1018.739136,6.287130,66.370613,19.440001,...,4,0,6,5.0,14.650000,5.0,11.950000,5.0,9.950000,16
16892,2023-01-25 07:00:00+00:00,18.250000,24.311678,0.0,0.0,1018.900024,1017.945007,5.400000,126.869980,16.919998,...,4,0,7,5.0,16.750000,5.0,14.650000,5.0,11.950000,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3140,2025-01-21 22:00:00+00:00,18.400000,75.788086,0.0,0.0,1013.599976,1012.650269,8.528657,315.855011,15.840000,...,4,0,22,5.0,16.358500,5.0,18.600000,5.0,16.908501,17192
16691,2025-01-22 03:00:00+00:00,16.258501,85.000000,0.0,0.0,1015.400024,1014.441650,3.826853,48.814175,6.840000,...,4,0,3,5.0,16.258501,5.0,16.008499,5.0,16.008499,17197
3852,2025-01-22 18:00:00+00:00,19.558500,64.000000,0.0,0.0,1016.400024,1015.451599,4.024922,26.564985,12.959999,...,4,0,18,5.0,19.558500,5.0,19.658501,5.0,19.658501,17212
16080,2025-01-22 19:00:00+00:00,19.258501,66.000000,0.0,0.0,1016.299988,1015.350647,4.104631,52.124958,7.559999,...,4,0,19,5.0,19.258501,5.0,19.558500,5.0,19.558500,17213


In [336]:
# Drop the 'datetime' column from the training features DataFrame 'X_train'
X_train.drop(["date", 'index'], axis=1, inplace=True)

# Drop the 'datetime' column from the test features DataFrame 'X_test'
X_test.drop(["date", 'index'], axis=1, inplace=True)

In [337]:
X_train.columns

Index(['temperature_2m', 'relative_humidity_2m', 'precipitation', 'rain',
       'pressure_msl', 'surface_pressure', 'wind_speed_10m',
       'wind_direction_10m', 'wind_gusts_10m', 'day_of_week', 'month',
       'day_of_year', 'week_of_year', 'is_weekend', 'hour', 'aqi_lag_1',
       'temp_lag_1', 'aqi_lag_2', 'temp_lag_2', 'aqi_lag_3', 'temp_lag_3'],
      dtype='object')

In [135]:
y_train.value_counts(normalize=True)

,proportion
main_aqi,
3,0.362158
5,0.275684
4,0.196617
2,0.150512
1,0.015029


In [338]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

# Initialize XGBRegressor
xgb_model = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=1,   # L1 regularization (try 1, 10, 100)
    reg_lambda=1,  # L2 regularization (try 1, 10, 100)
    random_state=42
)

# Fit the model
xgb_model.fit(X_train, y_train)

# Evaluate the model
y_pred = xgb_model.predict(X_test)

# Calculate R² and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"XGBoost R² Score: {r2}")
print(f"XGBoost Mean Squared Error: {mse}")

# Cross-validation for robustness
# Ensure 'scoring' is a valid metric for XGBRegressor
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validated MSE: {-cv_scores.mean():.4f} ± {cv_scores.std():.4f}") # Output as positive MSE

XGBoost R² Score: 0.9245370657200538
XGBoost Mean Squared Error: 0.09141961495060143
Cross-validated MSE: 0.1010 ± 0.0146


In [339]:
mr = project.get_model_registry()

In [340]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'temperature_2m',
    'type': 'float32'},
   {'name': 'relative_humidity_2m', 'type': 'float32'},
   {'name': 'precipitation', 'type': 'float32'},
   {'name': 'rain', 'type': 'float32'},
   {'name': 'pressure_msl', 'type': 'float32'},
   {'name': 'surface_pressure', 'type': 'float32'},
   {'name': 'wind_speed_10m', 'type': 'float32'},
   {'name': 'wind_direction_10m', 'type': 'float32'},
   {'name': 'wind_gusts_10m', 'type': 'float32'},
   {'name': 'day_of_week', 'type': 'int32'},
   {'name': 'month', 'type': 'int32'},
   {'name': 'day_of_year', 'type': 'int32'},
   {'name': 'week_of_year', 'type': 'int64'},
   {'name': 'is_weekend', 'type': 'int64'},
   {'name': 'hour', 'type': 'int32'},
   {'name': 'aqi_lag_1', 'type': 'float64'},
   {'name': 'temp_lag_1', 'type': 'float32'},
   {'name': 'aqi_lag_2', 'type': 'float64'},
   {'name': 'temp_lag_2', 'type': 'float32'},
   {'name': 'aqi_lag_3', 'type': 'float64'},
   {'name': 'temp_lag_3', 't

In [341]:
import os
import joblib

model_dir="aqi_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(xgb_model, model_dir + '/xgboost_pipeline.pkl')


['aqi_model/xgboost_pipeline.pkl']

In [342]:
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# calculate RMSE using sklearn
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

# calculate R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

MSE: 0.09141961495060143
RMSE: 0.30235676766131997
R squared: 0.9245370657200538


In [343]:
aq_model = mr.python.create_model(
    name="aqi_xgboost_model",
    metrics={
        "RMSE": rmse,
        "MSE": mse,
        "R squared": r2
    },
    model_schema=model_schema,
    input_example=X_test.sample().values,
    description="AQI predictor.")

aq_model.save(model_dir)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/427884 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/246 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1728 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1209521/models/aqi_xgboost_model/2


Model(name: 'aqi_xgboost_model', version: 2)